In [2]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from scipy import stats
import math

In [40]:
initial = pd.read_csv('C:/users/rusty/documents/github/patchseq_gdoc_analysis/patch_seq_log_mouse.csv')

In [41]:
ps = initial[['Date', 'Morph', 'Post patch?', 'Post patch pipette R', 'SM_QC_PF', 'Bad dates']]
ps['Date'] = pd.to_datetime(ps['Date'], format = '%y%m%d')
ps['Morph'] = ps['Morph'].astype('category')
#ps['Bad dates'] = ps['Bad dates'].astype('category')
#ps['PCR cycles'] = ps['PCR cycles'].astype('category')
ps['Post patch?'] = ps['Post patch?'].astype('category')
ps['SM_QC_PF'] = ps['SM_QC_PF'].astype('category')
ps['Bad dates'] = ps['Bad dates'].astype('category')
#ps = ps[ps['PCR cycles'] == 21]
ps = ps[ps['Bad dates'] != 'x']
ps = ps[ps['SM_QC_PF'] != 'cnt']
ps = ps[(ps['Morph'] == 'y') | (ps['Morph'] == 'n') | (ps['Morph'] == 'q')]
ps.dropna(subset=['Morph', 'Post patch?'], inplace=True)
ps

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

,Date,Morph,Post patch?,Post patch pipette R,SM_QC_PF,Bad dates
251,2016-03-18,n,nr,NaN,fail,NaN
252,2016-03-18,n,nr,NaN,fail,NaN
253,2016-03-18,y,nr,NaN,fail,NaN
254,2016-03-18,y,Outside-Out,NaN,pass,NaN
255,2016-03-18,n,nr,NaN,fail,NaN
256,2016-03-18,n,nr,NaN,fail,NaN
257,2016-03-18,y,nr,NaN,pass,NaN
258,2016-03-18,n,nr,NaN,pass,NaN
259,2016-03-18,n,nr,NaN,fail,NaN
265,2016-04-06,n,nr,NaN,fail,NaN


In [42]:
ps = ps.drop('Bad dates', 1)
ps = ps.drop('SM_QC_PF', 1)
ps.head()

,Date,Morph,Post patch?,Post patch pipette R
251,2016-03-18,n,nr,NaN
252,2016-03-18,n,nr,NaN
253,2016-03-18,y,nr,NaN
254,2016-03-18,y,Outside-Out,NaN
255,2016-03-18,n,nr,NaN


In [43]:
#ps['Post patch?'].value_counts()

In [44]:
def nucleated(x):
    nucleus = x[(x["Post patch?"] == "Nucleated") | (((x["Post patch?"] == "nucleus_visible") | (x["Post patch?"] == "nucleus_present")) & (x["Post patch pipette R"] >= 500))]
    return nucleus

def partial_nucleated(y):
    partial = y[(y["Post patch?"] == 'Partial-Nucleus') | (((y['Post patch?'] == 'nucleus_present') | (y['Post patch?'] == 'nucleus_visible')) & (y["Post patch pipette R"] <= 499))]
    return partial

def outside_out(z):
    outside = z[(z["Post patch?"] == 'Outside-Out') | (((z['Post patch?'] == 'nucleus_absent') | (z['Post patch?'] == 'no_nuclues_visible')) & (z['Post patch pipette R'] >= 500))]
    return outside

def no_seal(w): 
    no = w[(w["Post patch?"] == 'No-Seal') | (((w['Post patch?'] == 'nucleus_absent') | (w['Post patch?'] == 'no_nucleus_visible')) & (w['Post patch pipette R'] <= 499))]
    return no
    
def entire_cell(v):
    entire = v[(v["Post patch?"] == 'Entire-Cell') | (v['Post patch?'] == 'entire_cell')]
    return entire

In [45]:
nuc = nucleated(ps)
nuc["post patch class"] = "Nucleated"
part_nuc = partial_nucleated(ps)
part_nuc["post patch class"] = "Partial nucleated"
out_out = outside_out(ps)
out_out["post patch class"] = "Outside out"
noSeal = no_seal(ps)
noSeal["post patch class"] = "No seal"
entirecell = entire_cell(ps)
entirecell['post patch class'] = "Entire cell"

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [46]:
frames = [nuc, part_nuc, out_out, noSeal, entirecell]
ps = pd.concat(frames)
ps.head()

,Date,Morph,Post patch?,Post patch pipette R,post patch class
268,2016-04-06,y,Nucleated,NaN,Nucleated
280,2016-04-07,n,Nucleated,NaN,Nucleated
294,2016-04-08,n,Nucleated,NaN,Nucleated
303,2016-04-11,y,Nucleated,NaN,Nucleated
320,2016-04-12,y,Nucleated,NaN,Nucleated


In [47]:
ps = ps.drop('Post patch?', 1)
ps = ps.drop('Post patch pipette R', 1)
ps = ps.drop('Date', 1)
ps.head()

,Morph,post patch class
268,y,Nucleated
280,n,Nucleated
294,n,Nucleated
303,y,Nucleated
320,y,Nucleated


In [48]:
y = ps[ps['Morph'] == 'y']
y["morpho_call"] = "y"
n = ps[ps['Morph'] == 'n']
n['morpho_call'] = 'n'
q = ps[ps['Morph'] == 'q']
q['morpho_call'] = 'q'
#frames = [yes, no, quest]
#morphpp = pd.concat(frames)
#morphpp = morphpp.drop('Date', 1)
#morphpp
#ps['Morph'].value_counts()

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda2\lib\site-packages\ipykernel\__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [49]:
yes = y.groupby(['morpho_call', 'post patch class']).count()
yes = yes.reset_index()
yes = yes.pivot_table(values = 'Morph', columns = 'post patch class')

Morph
morpho_call post patch class        
y           Entire cell            7
            No seal              113
            Nucleated            424
            Outside out          432
            Partial nucleated     89